In [27]:
import os
import torch
import numpy as np
from src.utils import config
from src.loaddataset import load_dataset_for_inference
from src.buildmodel import build_model
from src.utils.modelfile import load_model

from stats.confusion_matrix import get_confusion_matrix, plot_confusion_matrix

def build_and_load_model(parameters, model_file ):
    model, device = build_model(parameters)
    load_model(parameters=parameters, model=model, model_file=model_file+'.pth')  
    model.eval()
    return model, device
    
def inference(model, device, data_file, batch_size = 600):

    test_loader = load_dataset_for_inference(data_file=data_file,  batch_size=batch_size)

    # Iterate over batches and evaluate the model
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            x, target = data
            x = x.to(device)
            out = model(x)
            # Perform inference
            #x_data = torch.cat((x_data, x), 0)  if i != 0 else x
            #y_data = torch.cat((y_data, target), 0)  if i != 0 else target
            #out_data = torch.cat((out_data, out ), 0) if i != 0 else out
            x = x.cpu().numpy()
            target = target.cpu().numpy()
            out = out.cpu().numpy()
            #x_data = np.concatenate((x_data, x), axis=0) if i != 0 else x
            y_data = np.concatenate((y_data, target), axis=0) if i != 0 else target
            out_data = np.concatenate((out_data, out), axis=0) if i != 0 else out

    return y_data, out_data

def get_data_file(dataset, parameters):
    match dataset:
        case 'train':
            return parameters.dataset.train_file
        case 'validation':
            return parameters.dataset.validation_file
        case _:
            return parameters.dataset.test_file
    
def save_output(config_file, dataset, result_data):
    folder = 'output/data'
    if not os.path.exists(folder):
        os.makedirs(folder)
    np.save(os.path.join(folder, config_file + '_' + dataset + '.npy'), result_data)

In [28]:
runs = 10
model_name = 'only_masks'
config_file = 'cave'

parameters = config.load_parameters(config_file)
for dataset in ['train', 'validation', 'test']:
  print(' **** dataset ' + dataset + ' *****')
  result_data = np.empty((runs,))
  for run in range(1, runs + 1):
    print(' **** run # ' + str(run) + ' *****')
    model, device = build_and_load_model(parameters=parameters, model_file=model_name+ '_run' + str(run))
    _, out_data = inference(model=model, device=device, data_file=get_data_file(dataset, parameters))
    #print('x_data.shape', x_data.shape)
    #print('y_data.shape', y_data.shape)
    if run == 1:
        result_data = np.empty((runs, out_data.shape[0], out_data.shape[1]))
    result_data[run-1, :, :] = out_data
    #np.stack((y_data, outputs), axis=2)
  save_output(config_file + '_' + model_name, dataset, result_data)


 **** dataset train *****
 **** run # 1 *****
Loading model ... Best epoch (53), Min loss (0.5036070200346284) 
 **** run # 2 *****
Loading model ... Best epoch (61), Min loss (0.5027891559115911) 
 **** run # 3 *****
Loading model ... Best epoch (46), Min loss (0.5068717699939922) 
 **** run # 4 *****
Loading model ... Best epoch (46), Min loss (0.5057908369828079) 
 **** run # 5 *****
Loading model ... Best epoch (49), Min loss (0.5067170019877159) 
 **** run # 6 *****
Loading model ... Best epoch (61), Min loss (0.4994514473414017) 
 **** run # 7 *****
Loading model ... Best epoch (51), Min loss (0.497221813363544) 
 **** run # 8 *****
Loading model ... Best epoch (71), Min loss (0.49993308974524675) 
 **** run # 9 *****
Loading model ... Best epoch (35), Min loss (0.5025398569591975) 
 **** run # 10 *****
Loading model ... Best epoch (60), Min loss (0.49545190495959784) 
 **** dataset validation *****
 **** run # 1 *****
Loading model ... Best epoch (53), Min loss (0.50360702003462

In [26]:
import numpy as np

# Assuming y_data and outputs are NumPy arrays with shape (1425, 2)
y_data = np.random.choice([0, 1], size=(200, 2))
outputs = np.random.rand(200, 2)
print(y_data.shape)
print(outputs.shape)
print(y_data[0])
print(outputs[0])
print(y_data[0][0])
print(outputs[0][0])
print(y_data[0][1])
print(outputs[0][1])

# Stack y_data and outputs along a new axis
new_array = np.stack((y_data, outputs), axis=2)
print('NEW ====')
print(new_array[0])
print(new_array[0][0])
print(new_array[0][1])
print(new_array.shape)


(200, 2)
(200, 2)
[1 1]
[0.88018877 0.96000449]
1
0.880188768136165
1
0.9600044858828005
NEW ====
[[1.         0.88018877]
 [1.         0.96000449]]
[1.         0.88018877]
[1.         0.96000449]
(200, 2, 2)
